# Training model

In [1]:
#Import Packages
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
import tqdm

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

df = pd.read_csv('Boliga - Final for training.csv')
df = df.drop(columns = ['Kommune'])
df = df.sample(500)

In [2]:
#Split into X and Y 
df_dum = pd.get_dummies(df, drop_first = True, columns = ['isForeclosure', 'Type'])
X = df_dum.drop(columns = ['price'])
y = df_dum['price']

In [3]:
pd.options.display.max_columns= 100
df_dum.head()

,price,basementSize,buildYear,Ejerudgift,lotSize,rooms,size,Relativ Ledighed,Grundskole,Gymnasiale uddannelser,Erhvervsfaglige uddannelser,KVU,MVU,Bacheloruddannelser,LVU,Kommunal_gennemsnitsinkomst_2017,Total_reported,Population_in_urban_development,Socioeconomic_index,expenses_per_school_student,average_class_size,expenses_sport_and_other_cultural_activities,lake_distance,forest_distance,doctor_distance,supermarket_distance,school_distance,daycare_distance,hospital_distance,train_distance,pharmacy_distance,library_distance,coast_distance,junction_distance,Type_Ejerlejlighed,Type_Fritidsgrund,Type_Fritidshus,Type_Helårsgrund,Type_Landejendom,Type_Rækkehus,Type_Villa,Type_Villalejlighed
58276,1495000,86,1939,1597,7349,6.0,239,5.355900,19.469621,3.969693,41.756115,6.234597,18.003137,0.810640,7.591095,309582,0.546043,80.1,0.90,69688,21.2,1153,4.29315,4.79861,4.48354,7.62335,2.68897,2.68897,18.83561,18058.30,13.22875,4.41272,13.83549,14.50697,0,0,0,0,0,0,1,0
3008,1450000,0,1945,2408,0,2.0,51,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.38604,2.14616,0.20286,0.09929,0.28703,0.26083,2.43037,2484.71,0.60324,1.90186,3.04362,1.26780,1,0,0,0,0,0,0,0
46789,1110000,0,1969,1628,0,2.0,53,7.030325,13.767009,6.063049,26.126501,5.889466,21.376051,2.236424,19.739691,314040,0.501700,85.3,0.83,64340,21.6,1500,1.67774,0.17444,0.61224,0.57429,0.90378,0.84330,7.66559,469.32,0.58002,0.84237,5.05563,1.67777,1,0,0,0,0,0,0,0
3408,4750000,0,1932,3561,792,6.0,161,10.778659,27.374678,6.543681,36.738814,5.584838,12.930358,1.093417,7.182909,283280,1.219234,100.0,1.06,82679,23.2,1674,3.29596,3.03477,1.41370,0.90446,0.94613,0.40429,2.19556,8855.76,0.95109,1.68070,5.29095,1.92726,0,0,0,0,0,0,1,0
47000,1045000,0,0,1390,846,0.0,0,5.951429,18.189920,4.883918,41.865336,5.779600,18.221113,1.332383,7.833875,309370,0.540793,88.1,1.02,66463,21.7,1225,6.70703,0.80987,4.10817,3.51909,3.69898,2.90364,20.21900,4283.05,3.53483,4.25834,2.39223,4.90988,0,0,0,1,0,0,0,0


In [4]:
pd.options.display.max_columns = 100
X.head()

,basementSize,buildYear,Ejerudgift,lotSize,rooms,size,Relativ Ledighed,Grundskole,Gymnasiale uddannelser,Erhvervsfaglige uddannelser,KVU,MVU,Bacheloruddannelser,LVU,Kommunal_gennemsnitsinkomst_2017,Total_reported,Population_in_urban_development,Socioeconomic_index,expenses_per_school_student,average_class_size,expenses_sport_and_other_cultural_activities,lake_distance,forest_distance,doctor_distance,supermarket_distance,school_distance,daycare_distance,hospital_distance,train_distance,pharmacy_distance,library_distance,coast_distance,junction_distance,Type_Ejerlejlighed,Type_Fritidsgrund,Type_Fritidshus,Type_Helårsgrund,Type_Landejendom,Type_Rækkehus,Type_Villa,Type_Villalejlighed
58276,86,1939,1597,7349,6.0,239,5.355900,19.469621,3.969693,41.756115,6.234597,18.003137,0.810640,7.591095,309582,0.546043,80.1,0.90,69688,21.2,1153,4.29315,4.79861,4.48354,7.62335,2.68897,2.68897,18.83561,18058.30,13.22875,4.41272,13.83549,14.50697,0,0,0,0,0,0,1,0
3008,0,1945,2408,0,2.0,51,7.926682,13.754726,7.646977,19.268325,4.928181,17.977495,3.627649,26.658495,319745,0.188050,100.0,1.08,69710,22.8,1062,1.38604,2.14616,0.20286,0.09929,0.28703,0.26083,2.43037,2484.71,0.60324,1.90186,3.04362,1.26780,1,0,0,0,0,0,0,0
46789,0,1969,1628,0,2.0,53,7.030325,13.767009,6.063049,26.126501,5.889466,21.376051,2.236424,19.739691,314040,0.501700,85.3,0.83,64340,21.6,1500,1.67774,0.17444,0.61224,0.57429,0.90378,0.84330,7.66559,469.32,0.58002,0.84237,5.05563,1.67777,1,0,0,0,0,0,0,0
3408,0,1932,3561,792,6.0,161,10.778659,27.374678,6.543681,36.738814,5.584838,12.930358,1.093417,7.182909,283280,1.219234,100.0,1.06,82679,23.2,1674,3.29596,3.03477,1.41370,0.90446,0.94613,0.40429,2.19556,8855.76,0.95109,1.68070,5.29095,1.92726,0,0,0,0,0,0,1,0
47000,0,0,1390,846,0.0,0,5.951429,18.189920,4.883918,41.865336,5.779600,18.221113,1.332383,7.833875,309370,0.540793,88.1,1.02,66463,21.7,1225,6.70703,0.80987,4.10817,3.51909,3.69898,2.90364,20.21900,4283.05,3.53483,4.25834,2.39223,4.90988,0,0,0,1,0,0,0,0


### Split data into Test, Development, Validation and Train

In [5]:
from sklearn.model_selection import train_test_split

X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size = 0.5, random_state = 1)

In [6]:

#corr = df_dum.corr()
#plt.figure(figsize=(14, 10))
#ax = sns.heatmap(
#    corr, 
 #   vmin=-1, vmax=1, center=0,
  #  cmap=sns.diverging_palette(20, 220, n=200),
   # square=True
#)
#ax.set_xticklabels(
 #   ax.get_xticklabels(),
  #  rotation=45,
   # horizontalalignment='right'
#);

In [7]:
#g = sns.pairplot(df.drop(columns = ['isForeclosure', 'Type']))

### Scale, fit and transform data

In [8]:
from sklearn.preprocessing import StandardScaler

#stdscale = StandardScaler(with_mean = 1, with_std = True)

#Scale explanatory variables
#X_train_s = stdscale.fit_transform(X_train)
#X_test_s = stdscale.transform(X_test)


#Scale target variable (because of regression model - no need when categorical)
#y_train_s = stdscale.fit_transform(y_train)
#y_test_s = stdscale.transform(y_test)


### Optimize hyperparameter with K-fold CV 

In [9]:
#Find optimized lambda
from sklearn.model_selection import KFold

lambdas =  np.logspace(-4, 4, 12) # define lambda - devide into 12 lambdas between 10**-4 and 10**4
kfolds = KFold(n_splits=5) # define nr. of folds

#Creates lists to append values of the loop:
mse_val_avr = []
mse_test_avr = []
lambda_list = []
mae_val_avr = []
mae_test_avr = []

#Outer-loop (Lambdas)
for lambda_ in tqdm.tqdm(lambdas):
#Inner-loop (Folds)
    mse_val_ = []
    mse_test_ = []
    mae_val_ = []
    mae_test_ = []    
    #Split the dataset into 5 folds, and loop through each fold. 
    # In the iteration, the fold in question acts as the training-set. 
    for train_idx,val_idx in kfolds.split(X_dev, y_dev):
    # train model and compute MSE on test fold
        pipe_Lasso_CV = make_pipeline(PolynomialFeatures(include_bias=False, degree = 3), 
                        StandardScaler(with_mean = 1, with_std = True ),
                        Lasso(alpha = lambda_, random_state=1))
        # Assign X_train & y_train by extracting elements if the folds from X_dev and y_dev
        X_train2, y_train2 = X_dev.iloc[train_idx], y_dev.iloc[train_idx]
        # Assign X_val & y_val by extracting elements of the folds from X_dev and y_dev
        X_val2, y_val2 = X_dev.iloc[val_idx], y_dev.iloc[val_idx] 
        # Fit Lasso-model to training data
        pipe_Lasso_CV.fit(X_train2, y_train2)   
        #Calculate and append the mse into mse_val list:
        mse_val_.append(mse(pipe_Lasso_CV.predict(X_val2), y_val2))
        ##Calculate and append the mse into mse_test list:
        mse_test_.append(mse(pipe_Lasso_CV.predict(X_test), y_test))
        #Calculate and append the mse into mse_val list:
        mae_val_.append(mae(pipe_Lasso_CV.predict(X_val2), y_val2))
        ##Calculate and append the mse into mse_test list:
        mae_test_.append(mae(pipe_Lasso_CV.predict(X_test), y_test))
        
        
        
    mse_val_avr.append(sum(mse_val_)/len(mse_val_))
    mse_test_avr.append(sum(mse_val_)/len(mse_val_))
    #mse_val.append(mse_val_)
    #mse_test.append(mse_test_)
    mae_val_avr.append(sum(mae_val_)/len(mae_val_))
    mae_test_avr.append(sum(mae_val_)/len(mae_val_))

df_val = pd.DataFrame(mse_val_avr, index = lambdas)
df_test = pd.DataFrame(mse_test_avr, index = lambdas)
df_join = pd.concat([df_test, df_val], axis=1)
df_with_index = df_join.reset_index()

df_val2 = pd.DataFrame(mae_val_avr, index = lambdas)
df_test2 = pd.DataFrame(mae_test_avr, index = lambdas)
df_join2 = pd.concat([df_test2, df_val2], axis=1)
df_with_index2 = df_join2.reset_index()

#Convert to list
mses = df_with_index.values.tolist()
maes = df_with_index.values.tolist()


 50%|█████     | 6/12 [00:31<00:31,  5.28s/it]/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
 58%|█████▊    | 7/12 [00:39<00:30,  6.18s/it]/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:4

In [10]:
pd.options.display.float_format = "{:.2f}".format

In [11]:
df_join

,0,0
0.00,462182332639159127113728.00,462182332639159127113728.00
0.00,462170987680834214952960.00,462170987680834214952960.00
0.00,462110445246167816077312.00,462110445246167816077312.00
0.02,461787427761444188323840.00,461787427761444188323840.00
0.08,460064372931931853029376.00,460064372931931853029376.00
0.43,450914383449345027473408.00,450914383449345027473408.00
2.31,403267233662856532590592.00,403267233662856532590592.00
12.33,48486570705905443143680.00,48486570705905443143680.00
65.79,2452881351313765376.00,2452881351313765376.00
351.12,265794832083118848.00,265794832083118848.00


In [12]:
df_join2

,0,0
0.00,34016896318.06,34016896318.06
0.00,34016479141.27,34016479141.27
0.00,34014252788.40,34014252788.40
0.02,34002371862.57,34002371862.57
0.08,33938925754.65,33938925754.65
0.43,33599997230.60,33599997230.60
2.31,31776670460.38,31776670460.38
12.33,11038228373.34,11038228373.34
65.79,103375424.99,103375424.99
351.12,37625851.52,37625851.52
